### LCEL
LCEL (LangChain Expression Language) is a concise, chainable syntax for composing LangChain components like prompts, models, retrievers, and output parsers in a pipeline. It simplifies building modular, readable, and reusable workflows for language model applications.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
# Loads environment variables from a .env file into your system's environment.
load_dotenv()
# os.environ is a dictionary in Python that holds all the environment variables available to your system during the program's execution.

# This is needed to enable LangSmith logging/tracing for chains, prompts, and LLM calls.
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

# Setting this to 'true' will start recording all your chain/agent/LLM activity for debugging and evaluation.
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

# This helps organize and filter traces inside the LangSmith dashboard based on this project.
os.environ['Langchain_Project'] = os.getenv('LANGCHAIN_PROJECT')

In [20]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [21]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(api_key=groq_api_key, model="Gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A2FE8999A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A2FE8D8B00>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

##### Without Using LCEL to inference LLM, Prompt and Stroutput Chains

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

response = model.invoke(messages)
print(response)

content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual way to greet a friend or someone you know well.)\n* **Bonjour, ça va ?** (This is a very common and informal way to greet someone.)\n\nLet me know if you\'d like to learn other ways to say hello in French!\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 21, 'total_tokens': 140, 'completion_time': 0.216363636, 'prompt_time': 0.002138284, 'queue_time': 0.23519330600000002, 'total_time': 0.21850192}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-e49a2bc9-c5f1-4e85-baff-de5b3926b647-0' usage_metadata={'input_tokens': 21, 'output_tokens': 119, 'total_tokens': 140}


In [23]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
output_response = parser.invoke(response)
print(output_response)

Here are a few ways to say "Hello, how are you?" in French:

**Formal:**

* **Bonjour, comment allez-vous ?** (This is the most formal way to greet someone.)

**Informal:**

* **Salut, comment vas-tu ?** (This is a more casual way to greet a friend or someone you know well.)
* **Bonjour, ça va ?** (This is a very common and informal way to greet someone.)

Let me know if you'd like to learn other ways to say hello in French!



##### With Using LCEL to inference LLM, Prompt and Stroutput Chains

In [24]:
chain = (model|parser).invoke(messages)
chain

'Here are a couple of ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?**  (This is the most polite and formal way to ask.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is more casual and used with friends or people you know well.)\n\n\nLet me know if you\'d like to translate anything else! 😊 \n'

#### Using PromptTemplate with LCEL : An efficient technique of LCEL

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = "You are a French language tutor. Translate the following sentence in {language}:"

prompt = ChatPromptTemplate(
    [
        ("system", template),
        ("user", "{text}")
    ]
)

In [31]:
chain = prompt | model | parser
result = chain.invoke({"language":"French", "text":"Hello How are you?"})
result

'Here are a couple of ways to say "Hello, how are you?" in French, along with explanations:\n\n**1. Bonjour, comment allez-vous?**\n\n* **Bonjour:** This means "Hello" and is a formal greeting. It\'s appropriate for most situations, but especially with strangers or people you respect.\n* **Comment allez-vous?:** This means "How are you?" and is the formal way to ask. It\'s polite and respectful.\n\n**2. Salut, ça va?**\n\n* **Salut:** This is a more informal "Hello," similar to "Hi" or "Hey."\n* **Ça va?:** This means "Is it going well?" or "How\'s it going?" It\'s a casual way to ask how someone is doing.\n\n**Which one to use?**\n\n* Use "Bonjour, comment allez-vous?" in formal situations or when meeting someone for the first time.\n\n* Use "Salut, ça va?" with friends, family, or people you know well.\n\n\n\nLet me know if you have any other French phrases you\'d like me to translate!\n'

In [32]:
print(result)

Here are a couple of ways to say "Hello, how are you?" in French, along with explanations:

**1. Bonjour, comment allez-vous?**

* **Bonjour:** This means "Hello" and is a formal greeting. It's appropriate for most situations, but especially with strangers or people you respect.
* **Comment allez-vous?:** This means "How are you?" and is the formal way to ask. It's polite and respectful.

**2. Salut, ça va?**

* **Salut:** This is a more informal "Hello," similar to "Hi" or "Hey."
* **Ça va?:** This means "Is it going well?" or "How's it going?" It's a casual way to ask how someone is doing.

**Which one to use?**

* Use "Bonjour, comment allez-vous?" in formal situations or when meeting someone for the first time.

* Use "Salut, ça va?" with friends, family, or people you know well.



Let me know if you have any other French phrases you'd like me to translate!

